In [ ]:
from nbdev import *
default_exp optimizer

In [ ]:
#hide
import sys
sys.path.append("..")
import json
import numpy as np
import matplotlib.pyplot as plt
import optuna
from dpct.individual import DHPCTIndividual

# DHPCTOptimizer

> Class for hyperparameter optimization of DHPCT models using Optuna.

In [ ]:
#exportclass DHPCTOptimizer:    """DHPCTOptimizer provides hyperparameter optimization capabilities for DHPCTIndividual instances using Optuna."""        def __init__(self, env_name, env_props=None, n_trials=100, timeout=None, db_storage=None):        """        Initialize a new optimizer with optimization parameters.                Parameters:        - env_name: Name of the OpenAI Gym environment to use        - env_props: Dictionary of environment properties        - n_trials: Number of trials to run        - timeout: Seconds to run optimization (None = no limit)        - db_storage: SQLite database file for storage (None = in-memory)        """                self.env_name = env_name        self.env_props = env_props or {}        self.n_trials = n_trials        self.timeout = timeout        self.db_storage = db_storage        self.study = None        self.best_params = None        self.evaluation_episodes = 10  # Default, can be changed        def define_objective(self, trial):        """        Define the objective function for Optuna.                Parameters:        - trial: Optuna trial object                Returns:        - Fitness (average reward)        """        # Define hyperparameters to optimize        n_levels = trial.suggest_int('n_levels', 1, 4)                # Create lists to store level configurations        levels = []        activation_funcs = []        weight_types = []                # Define parameters for each level        for i in range(n_levels):            # Units for this level            units = trial.suggest_int(f'level_{i}_units', 4, 64)            levels.append({'units': units})                        # Activation function for this level            activation_options = ['linear', 'relu', 'tanh', 'sigmoid']            activation = trial.suggest_categorical(f'level_{i}_activation', activation_options)            activation_funcs.append(activation)                        # Weight initialization for this level            weight_options = ['glorot_uniform', 'glorot_normal', 'he_uniform', 'he_normal']            weight_type = trial.suggest_categorical(f'level_{i}_weight', weight_options)            weight_types.append(weight_type)                # Create and evaluate individual        individual = DHPCTIndividual(            env_name=self.env_name,            env_name=self.gym_name,            env_props=self.env_props,            levels=levels,            activation_funcs=activation_funcs,            weight_types=weight_types        )                # Evaluate the individual        fitness = individual.evaluate(episodes=self.evaluation_episodes)        return fitness        def run_optimization(self, evaluation_episodes=None, verbose=True):        """        Run the hyperparameter optimization.                Parameters:        - evaluation_episodes: Number of episodes for individual evaluation        - verbose: Whether to print progress                Returns:        - Optuna study object with optimization results        """        # Set evaluation episodes if provided        if evaluation_episodes is not None:            self.evaluation_episodes = evaluation_episodes                # Create study storage        if self.db_storage is not None:            storage = f"sqlite:///{self.db_storage}"        else:            storage = None                # Create and run the optimization study        sampler = optuna.samplers.TPESampler(seed=42)  # For reproducibility        self.study = optuna.create_study(            storage=storage,            sampler=sampler,            study_name=f"dpct_{self.env_name}",            direction='maximize',            load_if_exists=True        )                self.study.optimize(            self.define_objective,            n_trials=self.n_trials,            timeout=self.timeout,            show_progress_bar=verbose        )                # Store best parameters        self.best_params = self.study.best_params                if verbose:            print(f"Best trial: {self.study.best_trial.number}")            print(f"Best value: {self.study.best_value}")            print(f"Best parameters: {self.best_params}")                return self.study        def get_best_individual(self):        """        Create a DHPCTIndividual with the best hyperparameters.                Returns:        - DHPCTIndividual with best parameters        """        if self.best_params is None:            raise ValueError("No optimization results available. Run optimization first.")                # Extract parameters        n_levels = self.best_params['n_levels']                # Create lists to store level configurations        levels = []        activation_funcs = []        weight_types = []                # Create level configurations from parameters        for i in range(n_levels):            units = self.best_params[f'level_{i}_units']            levels.append({'units': units})                        activation = self.best_params[f'level_{i}_activation']            activation_funcs.append(activation)                        weight_type = self.best_params[f'level_{i}_weight']            weight_types.append(weight_type)                # Create individual with best parameters        best_individual = DHPCTIndividual(            env_name=self.env_name,            env_name=self.gym_name,            env_props=self.env_props,            levels=levels,            activation_funcs=activation_funcs,            weight_types=weight_types        )                return best_individual        def visualize_results(self):        """        Visualize the optimization results.                Returns:        - List of matplotlib figures        """        if self.study is None:            raise ValueError("No optimization results to visualize. Run optimization first.")                figures = []                # Optimization history        fig = optuna.visualization.plot_optimization_history(self.study)        figures.append(fig)                # Parameter importances        try:            fig = optuna.visualization.plot_param_importances(self.study)            figures.append(fig)        except:            # Some versions or configurations might fail on this            pass                # Parallel coordinate plot for parameters        fig = optuna.visualization.plot_parallel_coordinate(self.study)        figures.append(fig)                return figures        def save_results(self, best_params_file="best_params.json", best_individual_file="best_individual.json"):        """        Save optimization results to files.                Parameters:        - best_params_file: File to save best parameters        - best_individual_file: File to save best individual configuration        """        if self.best_params is None:            raise ValueError("No optimization results to save. Run optimization first.")                # Save best parameters        with open(best_params_file, 'w') as f:            json.dump(self.best_params, f, indent=2)                # Save best individual        best_individual = self.get_best_individual()        best_individual.save_config(best_individual_file)

In [ ]:
# Example of using DHPCTOptimizer

# Initialize the optimizer
optimizer = DHPCTOptimizer(
    env_name="CartPole",
    env_name="CartPole-v1",
    env_props={},
    n_trials=20,  # Small number for quick example
    timeout=None,
    db_storage=None
)

In [ ]:
# Run optimization (commented out as it would take time)
# study = optimizer.run_optimization(evaluation_episodes=3, verbose=True)

In [ ]:
# Get best individual (commented out as it depends on optimization)
# best_individual = optimizer.get_best_individual()
# best_individual.compile()
# best_individual.model.summary()

In [ ]:
from nbdev import *
default_exp optimizer

In [ ]:
#hide
import sys
sys.path.append("..")
import json
import time
import numpy as np
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_contour
import matplotlib.pyplot as plt

In [ ]:
#hide
from dpct.individual import DHPCTIndividual
from dpct.evolver import DHPCTEvolver

# DHPCTOptimizer

> Class for optimizing hyperparameters of the evolution process using Optuna.

In [ ]:
#exportclass DHPCTOptimizer:    """DHPCTOptimizer uses Optuna to perform hyperparameter optimization on the evolution process."""        def __init__(self,                  evolution_params,                  n_trials=100,                  timeout=None,                  pruner=None,                  sampler=None,                 study_name=None,                 storage=None):        """        Initialize the optimizer with configuration parameters.                Parameters:        - evolution_params: Dictionary of parameters for evolution, each with a 'fixed' flag                          If fixed=True, the parameter is not optimized and the provided value is used                          If fixed=False, the parameter is optimized within the specified range        - n_trials: Number of optimization trials to run        - timeout: Maximum time for optimization in seconds        - pruner: Optuna pruner instance        - sampler: Optuna sampler instance        - study_name: Name for the Optuna study        - storage: Optuna storage URL        """        self.evolution_params = evolution_params        self.n_trials = n_trials        self.timeout = timeout        self.pruner = pruner or optuna.pruners.MedianPruner()        self.sampler = sampler or optuna.samplers.TPESampler()        self.study_name = study_name or "DPCT_optimization"        self.storage = storage                self.study = None        self._objective_func = None        self._template_individual = None        self._fitness_function = None        self._evaluation_budget = None        def define_objective(self, template_individual, fitness_function, evaluation_budget=None):        """Define the objective function for Optuna to optimize                Parameters:        - template_individual: A DHPCTIndividual to use as a template        - fitness_function: Function to evaluate individuals        - evaluation_budget: Maximum number of evaluations per trial        """        self._template_individual = template_individual        self._fitness_function = fitness_function        self._evaluation_budget = evaluation_budget                def objective(trial):            """Objective function for Optuna"""            # Extract parameters for this trial            params = {}                        for param_name, param_config in self.evolution_params.items():                if param_config.get("fixed", False):                    # Use the fixed value                    params[param_name] = param_config.get("value")                else:                    # Optimize this parameter                    param_type = param_config.get("type", "float")                    param_min = param_config.get("min")                    param_max = param_config.get("max")                    param_step = param_config.get("step")                    param_choices = param_config.get("choices")                                        if param_type == "float":                        params[param_name] = trial.suggest_float(param_name, param_min, param_max, step=param_step)                    elif param_type == "int":                        params[param_name] = trial.suggest_int(param_name, param_min, param_max, step=param_step or 1)                    elif param_type == "categorical":                        params[param_name] = trial.suggest_categorical(param_name, param_choices)                    else:                        raise ValueError(f"Unknown parameter type: {param_type}")                        # Create an evolver with the selected parameters            evolver = DHPCTEvolver(                pop_size=params.get("pop_size", 50),                generations=params.get("generations", 100),                evolve_termination=params.get("evolve_termination"),                evolve_static_termination=params.get("evolve_static_termination"),                unchanged_generations=params.get("unchanged_generations", 5),                run_best=params.get("run_best", False),  # Don't run during optimization                save_arch_best=params.get("save_arch_best", False),  # Don't save during optimization                save_arch_all=params.get("save_arch_all", False)  # Don't save during optimization            )                        # Apply hierarchy parameters to the template individual if specified            individual_template = DHPCTIndividual(self._template_individual.gym_name,                self._template_individual.env_props.copy(),                params.get("levels", self._template_individual.levels[:]),                self._template_individual.activation_funcs.copy(),                self._template_individual.weight_types.copy()            )                        # Set up the evolution            evolver.setup_evolution(                individual_template,                 self._fitness_function,                 minimize=params.get("minimize", True)            )                        # Run the evolution with limited generations for optimization            generations_limit = min(                params.get("generations", 100),                self._evaluation_budget or params.get("generations", 100)            )            evolver.generations = generations_limit                        # Run the evolution silently (no verbose output)            population, logbook, hof = evolver.run_evolution(verbose=False)                        # Return the best fitness (minimum or maximum depending on the problem)            if params.get("minimize", True):                return min(ind.fitness.values[0] for ind in population)            else:                return max(ind.fitness.values[0] for ind in population)                self._objective_func = objective        return self        def run_optimization(self, verbose=True):        """Run the hyperparameter optimization process                Parameters:        - verbose: Whether to print progress information                Returns:        - The Optuna study object        """        if self._objective_func is None:            raise ValueError("Objective function not defined. Call define_objective first.")                # Create the study        self.study = optuna.create_study(            study_name=self.study_name,            storage=self.storage,            sampler=self.sampler,            pruner=self.pruner,            direction="minimize" if self.evolution_params.get("minimize", {}).get("value", True) else "maximize",            load_if_exists=True        )                # Run the optimization        if verbose:            print(f"Starting optimization with {self.n_trials} trials...")            start_time = time.time()                self.study.optimize(            self._objective_func,             n_trials=self.n_trials,            timeout=self.timeout,            show_progress_bar=verbose        )                if verbose:            total_time = time.time() - start_time            print(f"Optimization completed in {total_time:.2f} seconds.")            print(f"Best trial: {self.study.best_trial.number}")            print(f"Best value: {self.study.best_value}")            print(f"Best parameters: {self.study.best_params}")                return self.study        def get_best_params(self):        """Get the best parameters from the optimization                Returns:        - Dictionary of optimized parameters        """        if self.study is None:            raise ValueError("No optimization has been run. Call run_optimization first.")                # Combine fixed parameters with optimized parameters        params = {}                for param_name, param_config in self.evolution_params.items():            if param_config.get("fixed", False):                params[param_name] = param_config.get("value")                # Add optimized parameters        params.update(self.study.best_params)                return params        def visualize_results(self):        """Visualize the optimization results                Returns:        - Dictionary of plot figures        """        if self.study is None:            raise ValueError("No optimization has been run. Call run_optimization first.")                # Create plots        plots = {}        plots["optimization_history"] = plot_optimization_history(self.study)        plots["param_importances"] = plot_param_importances(self.study)                # Get top 2 parameters for contour plot        if len(self.study.best_params) >= 2:            param_names = list(self.study.best_params.keys())            plots["contour"] = plot_contour(self.study, param_names[0], param_names[1])                return plots        def save_results(self, path):        """Save optimization results                Parameters:        - path: Base path to save results to        """        if self.study is None:            raise ValueError("No optimization has been run. Call run_optimization first.")                # Save best parameters        with open(f"{path}_best_params.json", "w") as f:            json.dump(self.get_best_params(), f, indent=2)                # Save trial history        trials = []        for trial in self.study.trials:            trial_dict = {                "number": trial.number,                "value": trial.value,                "params": trial.params,                "state": str(trial.state)            }            trials.append(trial_dict)                with open(f"{path}_trials.json", "w") as f:            json.dump(trials, f, indent=2)                # Save visualization plots        plots = self.visualize_results()        for name, fig in plots.items():            fig.write_image(f"{path}_{name}.png")

In [ ]:
# Example evolution parameters for optimization
evolution_params = {
    "pop_size": {"fixed": False, "type": "int", "min": 10, "max": 100, "step": 10},
    "generations": {"fixed": True, "value": 50},
    "evolve_static_termination": {"fixed": True, "value": True},
    "unchanged_generations": {"fixed": False, "type": "int", "min": 3, "max": 10},
    "minimize": {"fixed": True, "value": False}
}

# Create an optimizer
optimizer = DHPCTOptimizer(evolution_params, n_trials=5)